### Import libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

### Load and split data

In [2]:
ds_train = keras.utils.image_dataset_from_directory(
    directory = "../data/",
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(45, 45),
    shuffle=True,
    seed=0,
    validation_split=0.2,
    subset="training"
)

ds_val = keras.utils.image_dataset_from_directory(
    directory = "../data/",
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(45, 45),
    shuffle=True,
    seed=0,
    validation_split=0.2,
    subset="validation"
)

ds_test = ds_val.take(290)
ds_val = ds_val.skip(290)

Found 185465 files belonging to 16 classes.
Using 148372 files for training.


2022-04-21 21:02:40.785104: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-21 21:02:40.812386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-21 21:02:40.812573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-21 21:02:40.813101: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 185465 files belonging to 16 classes.
Using 37093 files for validation.


In [4]:
next(iter(ds_train))

(<tf.Tensor: shape=(64, 45, 45, 3), dtype=float32, numpy=
 array([[[[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.]],
 
         [[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.]],
 
         [[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.]],
 
         ...,
 
         [[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.]],
 
         [[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],

In [32]:
print('Batches for testing -->', ds_test.cardinality())
print('Batches for validating -->', ds_val.cardinality())

Batches for testing --> tf.Tensor(290, shape=(), dtype=int64)
Batches for validating --> tf.Tensor(290, shape=(), dtype=int64)


### Create model

In [33]:
model = keras.Sequential(
    [
        layers.Conv2D(filters=32, kernel_size=5, strides=1, activation="relu", input_shape=(45, 45, 3), kernel_regularizer=regularizers.l2(0.0005)),
        layers.Conv2D(filters=32, kernel_size=5, strides=1, use_bias=False),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.MaxPooling2D(pool_size=2, strides=2),
        layers.Dropout(0.25),
        layers.Conv2D(filters=64, kernel_size=3, strides=1, activation="relu", kernel_regularizer=regularizers.l2(0.0005)),
        layers.Conv2D(filters=64, kernel_size=3, strides=1, use_bias=False),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.MaxPooling2D(pool_size=2, strides=2),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(units=256, use_bias=False),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Dense(units=128, use_bias=False),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Dense(units=64, use_bias=False),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Dropout(0.25),
        layers.Dense(units=16, activation="softmax")
    ]
)

### Compile model

In [34]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

### Train model

In [36]:
model.fit(ds_train, epochs=30, validation_data=ds_val)

Epoch 1/30
2319/2319 [==============================] - 21s 9ms/step - loss: 0.0428 - accuracy: 0.9877 - val_loss: 0.0635 - val_accuracy: 0.9818
Epoch 2/30
2319/2319 [==============================] - 21s 9ms/step - loss: 0.0408 - accuracy: 0.9884 - val_loss: 0.0804 - val_accuracy: 0.9752
Epoch 3/30
2319/2319 [==============================] - 21s 9ms/step - loss: 0.0396 - accuracy: 0.9886 - val_loss: 0.8721 - val_accuracy: 0.7734
Epoch 4/30
2319/2319 [==============================] - 21s 9ms/step - loss: 0.0381 - accuracy: 0.9888 - val_loss: 1.2991 - val_accuracy: 0.5866
Epoch 5/30
2319/2319 [==============================] - 22s 9ms/step - loss: 0.0368 - accuracy: 0.9890 - val_loss: 0.0330 - val_accuracy: 0.9912
Epoch 6/30
2319/2319 [==============================] - 21s 9ms/step - loss: 0.0364 - accuracy: 0.9894 - val_loss: 0.8943 - val_accuracy: 0.7443
Epoch 7/30
2319/2319 [==============================] - 21s 9ms/step - loss: 0.0358 - accuracy: 0.9895 - val_loss: 0.8086 - val_ac

### Evaluate model on test set

In [37]:
model.evaluate(ds_test)

290/290 [==============================] - 1s 4ms/step - loss: 0.0204 - accuracy: 0.9950


[0.02037290297448635, 0.9949892163276672]

### Save model

In [39]:
model.save("../model")

2022-04-21 17:13:38.856782: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../model/assets
